In [1]:
# In data filtering we filter the duplicates data already, now we need to process the data into desired formats

In [1]:
import pandas as pd
import numpy as np
import datetime
import time

In [3]:
data = pd.read_csv('d_filtered.csv', low_memory=False, index_col=0)
data = data.drop(columns=['datetime date'])

C:\Users\Lance\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
# Record the starting and ending dates
cur_tic = data['PERMNO'][0]
start_ind = 0
dates_ind = {}
dates = {}

for i in range(len(data)):
    if data['PERMNO'][i]==cur_tic:
        pass
    else:
        dates_ind.update({cur_tic:[start_ind, i-1]})
        dates.update({cur_tic:[data['date'][start_ind],data['date'][i-1]]})
        start_ind=i
        cur_tic = data['PERMNO'][i]
# The last one is not updated
dates_ind.update({cur_tic:[start_ind, i]})
dates.update({cur_tic:[data['date'][start_ind],data['date'][i]]})

In [5]:
# Process the dividend
# Dividend is paid every 3 or 6 months, so we need to average the dividend payment to every month
data['DIVAMT'] = data['DIVAMT'].fillna(0)
div_lst = data['DIVAMT'].copy().values
total_length = len(dates)

count = 0
start_time = time.time()
for ticker in dates.keys():
    start_ind = 0
    tog = False # forward looking. tog is for whether first divdend is found
    div = 0
    for i in range(dates_ind[ticker][0], dates_ind[ticker][1]+1):
        if div_lst[i]==0:
            pass
        else:
            # The tog is False means you havent encountered the first dividend payment
            # i.e. This is the first dividend payment, and you will only run into this part of the loop once
            if tog==False: 
                div = div_lst[i]
                start_ind = i # where the dividend payment is
                tog = True # you have seen the first dividend, now everything gets started
                
            # when you encounter another div payment. now you smooth the div payment between new div and the last div
            else:
                div_split = div/(i-start_ind) # if the div is seasoanl, i-start_ind = 3
                div_lst[start_ind: i] = div_split
                start_ind = i
                div = div_lst[i]
                
    count+=1 #count every stock processed
    #if (count/total_length)%0.01<0.00001: # print every 1% done
    #    print('%d percent done. Time consumed: %d'%(count/total_length*100, time.time()-start_time))
a = data['DIVAMT'].values
len(div_lst[div_lst==0]), len(a[a==0])

(1866631, 3353995)

In [6]:
data['DIVAMT'] = div_lst
# a few more things to do:
# 1. Get markrt cap
data['Mkt Cap'] = data['SHROUT'] * data['PRC']
# 2. Get filtered dates that has missing price and missing sharesoutstanding, here you can just leave them as nan
# 3. Get the rolling vol
# 4. Get N-period momemtum

In [4]:
# We still need to align all the dates to the last day of the month
# Filter data, get when are the dates used
head, mid, tail = [], [], []
for i1 in range(1964, 2018):
    for i2 in range(2,13):
        mid.append(datetime.datetime(i1, i2, 1)-datetime.timedelta(days = 1))
    mid.append(datetime.datetime(i1+1, 1, 1)-datetime.timedelta(days = 1))

for i1 in range(8, 13):
    head.append(datetime.datetime(1963, i1, 1)-datetime.timedelta(days = 1))
head.append(datetime.datetime(1964, 1, 1)-datetime.timedelta(days = 1))
    
for i1 in range(2, 8):
    tail.append(datetime.datetime(2018,i1,1)-datetime.timedelta(days = 1))

dates_month = head+mid+tail # The index of the data file, in months

In [ ]:
# make sure the dates are aligned to the last day of the month
def match_last_day(date):
    date = datetime.datetime.strptime(date,'%Y/%m/%d')
    if date not in dates_month:
        if date.month<=11:
            return datetime.datetime(date.year, date.month+1, 1) - datetime.timedelta(days=1)
        else:
            return datetime.datetime(date.year+1, 1, 1) - datetime.timedelta(days=1)
    else:
        return date

data['date'] = data['date'].apply(match_last_day)
data

In [9]:
# 3. Get the rolling vol
# get_rolling_vol will record the rolling average of vol of the stock prices in a new column
def get_rolling_vol(rolling_vol_length): 
    volw = rolling_vol_length # rolling vol window
    rvols = [np.nan for i in range(len(data))] # the list of rolling vols, the first volw of the vols should are used as np.nan
    prices = data['PRC'].copy().values
    
    for ticker in dates_ind:
        si, ei = dates_ind[ticker][0], dates_ind[ticker][1]
        if ei-si>volw:
            for index in range(si+volw, ei+1):
                rvols[index] = np.std(prices[index-volw:index])
    return rvols


95.64783811569214


In [11]:
start_time = time.time()
rolling_12mvol = get_rolling_vol(12)
print(time.time()-start_time)
rolling_24mvol = get_rolling_vol(24)
print(time.time()-start_time)
rolling_36mvol = get_rolling_vol(36)
print(time.time()-start_time) # About 90s each

107.12406754493713
214.6107885837555
332.1202096939087


In [5]:
# 4.Get N-period Momentum
def get_momentum(mom_w):
    wind = mom_w
    mom_lst = [np.nan for i in range(len(data))]
    prices = data['PRC'].copy().values
    for ticker in dates_ind:
        si, ei = dates_ind[ticker][0], dates_ind[ticker][1]
        if ei-si>wind:
            for index in range(si+wind, ei+1):
                mom_lst[index] = (prices[index-1]-prices[index-wind])/prices[index-wind]
    return mom_lst

start_time = time.time()
mom_12m = get_momentum(12)
print(time.time()-start_time)
mom_24m = get_momentum(24)
print(time.time()-start_time)
mom_36m = get_momentum(36)
print(time.time()-start_time)

3.9716947078704834
7.581351280212402
10.934911727905273


In [17]:
data = data.drop(columns=['36m Vol','12m Mom'])

In [19]:
data['12m Vol'], data['24m Vol'], data['36m Vol'] = rolling_12mvol, rolling_24mvol, rolling_36mvol
data['12m Mom'], data['24m Mom'],data['36m Mom'] = mom_12m, mom_24m, mom_36m
data

,PERMNO,date,DIVAMT,PRC,SHROUT,Mkt Cap,12m Vol,24m Vol,36m Vol,12m Mom,24m Mom,36m Mom
0,10000,1985-12-31,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000,1986-01-31,0.000000,4.37500,3680.0,1.610000e+04,NaN,NaN,NaN,NaN,NaN,NaN
2,10000,1986-02-28,0.000000,3.25000,3680.0,1.196000e+04,NaN,NaN,NaN,NaN,NaN,NaN
3,10000,1986-03-31,0.000000,4.43750,3680.0,1.633000e+04,NaN,NaN,NaN,NaN,NaN,NaN
4,10000,1986-04-30,0.000000,4.00000,3793.0,1.517200e+04,NaN,NaN,NaN,NaN,NaN,NaN
5,10000,1986-05-31,0.000000,3.10938,3793.0,1.179388e+04,NaN,NaN,NaN,NaN,NaN,NaN
6,10000,1986-06-30,0.000000,3.09375,3793.0,1.173459e+04,NaN,NaN,NaN,NaN,NaN,NaN
7,10000,1986-07-31,0.000000,2.84375,3793.0,1.078634e+04,NaN,NaN,NaN,NaN,NaN,NaN
8,10000,1986-08-31,0.000000,1.09375,3793.0,4.148594e+03,NaN,NaN,NaN,NaN,NaN,NaN
9,10000,1986-09-30,0.000000,1.03125,3793.0,3.911531e+03,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
data.to_csv('with_vol_div.csv',sep=',')

In [2]:
data = pd.read_csv('with_vol_div.csv', low_memory=False, index_col=0)
data

C:\Users\Lance\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,PERMNO,date,DIVAMT,PRC,SHROUT,Mkt Cap,12m Vol,24m Vol,36m Vol,12m Mom,24m Mom,36m Mom
0,10000,1985-12-31,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000,1986-01-31,0.000000,4.37500,3680.0,1.610000e+04,NaN,NaN,NaN,NaN,NaN,NaN
2,10000,1986-02-28,0.000000,3.25000,3680.0,1.196000e+04,NaN,NaN,NaN,NaN,NaN,NaN
3,10000,1986-03-31,0.000000,4.43750,3680.0,1.633000e+04,NaN,NaN,NaN,NaN,NaN,NaN
4,10000,1986-04-30,0.000000,4.00000,3793.0,1.517200e+04,NaN,NaN,NaN,NaN,NaN,NaN
5,10000,1986-05-31,0.000000,3.10938,3793.0,1.179388e+04,NaN,NaN,NaN,NaN,NaN,NaN
6,10000,1986-06-30,0.000000,3.09375,3793.0,1.173459e+04,NaN,NaN,NaN,NaN,NaN,NaN
7,10000,1986-07-31,0.000000,2.84375,3793.0,1.078634e+04,NaN,NaN,NaN,NaN,NaN,NaN
8,10000,1986-08-31,0.000000,1.09375,3793.0,4.148594e+03,NaN,NaN,NaN,NaN,NaN,NaN
9,10000,1986-09-30,0.000000,1.03125,3793.0,3.911531e+03,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
len(dates_ind.keys())

32148

In [12]:
dates_ind

{10000: [0, 18],
 10001: [19, 399],
 10002: [400, 726],
 10003: [727, 847],
 10005: [848, 915],
 10006: [916, 1167],
 10007: [1168, 1226],
 10008: [1227, 1262],
 10009: [1263, 1442],
 10010: [1443, 1559],
 10011: [1560, 1677],
 10012: [1678, 1914],
 10013: [1915, 1933],
 10014: [1934, 2099],
 10015: [2100, 2135],
 10016: [2136, 2321],
 10017: [2322, 2355],
 10018: [2356, 2475],
 10019: [2476, 2674],
 10020: [2675, 2764],
 10021: [2765, 2886],
 10023: [2887, 2894],
 10024: [2895, 2972],
 10025: [2973, 3346],
 10026: [3347, 3736],
 10027: [3737, 3787],
 10028: [3788, 4177],
 10029: [4178, 4233],
 10030: [4234, 4299],
 10031: [4300, 4369],
 10032: [4370, 4759],
 10033: [4760, 4771],
 10034: [4772, 4840],
 10035: [4841, 5056],
 10036: [5057, 5119],
 10037: [5120, 5341],
 10038: [5342, 5402],
 10039: [5403, 5588],
 10040: [5589, 5630],
 10041: [5631, 5683],
 10042: [5684, 5950],
 10043: [5951, 6112],
 10044: [6113, 6502],
 10045: [6503, 6518],
 10046: [6519, 6629],
 10047: [6630, 6738],
 10